In [3]:
#packages

import nltk
nltk.download('punkt')
import re
from genderize import Genderize
from nltk.corpus import stopwords
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
import pickle
import pandas as pd

#name_probability_list = pd.read_pickle(r'C:\Users\DanielleDuncan\Documents\Masters-Thesis\name_probability_list.pickle')
name_probability_list = pd.read_pickle(r"/Users/yolandaferreirofranchi/Documents/GitHub/Masters-Thesis/name_probability_list.pickle")
#df_09 = pd.read_pickle(r'C:\Users\DanielleDuncan\Documents\Masters-Thesis\2009_preprocessed_date.pickle')
df_09 = pd.read_pickle(r"/Users/yolandaferreirofranchi/Documents/GitHub/Masters-Thesis/2009_preprocessed_date.pickle")


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yolandaferreirofranchi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#Ingestion json file

#filejson = r"C:\Users\danie\Desktop\bbc_latest_news_dataset_2021.json"

#article_df_4 = pd.read_json(filejson)
#article_df_4.info()

TO DO:
- break down the dataset files into years
- remove the name portion and have it seperate
- add two save points for preprocessed and training data
- add threading (safe list!!!!)
- finish the paper section

In [ ]:
#Ingestion

#Actual Dataset--File 1

filecsv = r"C:\Users\danie\Desktop\bbc_news_01.csv"
article_df_1 = pd.read_csv(filecsv)

#Actual Dataset--File 2

filecsv = r"C:\Users\danie\Desktop\bbc_news_02.csv"
article_df_2 = pd.read_csv(filecsv)

#Actual Dataset--File 3
filecsv = r"C:\Users\danie\Desktop\bbc_news_03.csv"
article_df_3 = pd.read_csv(filecsv)

#Merge Datasets

article_df = pd.concat([article_df_1, article_df_2, article_df_3])
article_df = article_df.assign(Article_Number=range(len(article_df)))
article_df = article_df.reset_index()
article_df.drop(["publisher", "header_image", "index", "raw_description", "short_description", "uniq_id", "scraped_at"], axis=1)
year = article_df['published_at'].str[:4]
article_df['year']=year

In [ ]:
article_df["year"].value_counts()

In [ ]:
# tokenize sentences in an article 

def split_sentences(article, article_id, year):
    pattern = r'(?<=[a-z0-9"]) *[.?!] *(?=[A-Z])'
    article = re.sub(pattern, r'\g<0> ', article)
    sentences = nltk.sent_tokenize(article)
    sentences_with_id = [(sentence, article_id, year) for sentence in sentences]
    return sentences_with_id

sentences_list = []

# add sentences to a new DF along with article ID 
for article, article_id, year in article_df[['description','Article_Number', 'year']].values:
    sentences = split_sentences(str(article), article_id, year)
    sentences_list.extend(sentences)

sentences_df = pd.DataFrame(sentences_list, columns= ['sentences', 'article_id', 'year'])

In [ ]:
df_12 = sentences_df[sentences_df['year'] == '2012']

In [ ]:
import nltk
nltk.download('omw-1.4')

In [ ]:
def PreProcessing (sentence):
    Male_count = 0
    Female_count = 0
    APIcallfail= 0

#regex_cleanup
    sentence = re.sub(r'https?:\/\/.*[\r\n]*', '', sentence)
    sentence = re.sub(r'\<a href', ' ', sentence)
    sentence = re.sub(r'&amp;', '', sentence) 
    sentence = re.sub("\d+", " ", sentence)
    sentence = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', sentence)
    sentence = re.sub(r'<br />', ' ', sentence)
    sentence = re.sub(r"\b's\b", '', sentence)

#tokenize
    sentence =  nltk.TweetTokenizer().tokenize(sentence)

#remove small words
    sentence = [ x for x in sentence if len(x) > 2 ]

#tag_and_stem
    tagged_sentence = nltk.tag.pos_tag(sentence)
    lemma = nltk.stem.WordNetLemmatizer()
    pn_tags = {'NNP', 'NNPS'}

    new_words = []
    proper_nouns = []

    for word, tag in tagged_sentence: 
        if tag not in pn_tags: 
            if tag.startswith("V"):
                lemmas = lemma.lemmatize(word, "v")
            else: 
                lemmas = lemma.lemmatize(word)
            new_words.append((lemmas))
        else:
            proper_nouns.append([word, tag])

    sentence = new_words

#name_gender
    #nltk_results = ne_chunk(tagged_sentence)
    nltk_results = ne_chunk(proper_nouns)

    for nltk_result in nltk_results:
        if type(nltk_result) == Tree:
            name = ''
            for nltk_result_leaf in nltk_result.leaves():
                name += nltk_result_leaf[0] + ' '
            if nltk_result.label() == "PERSON":
                name = name.split(' ')[0]
                try: 
                    word_gender = name_probability_list.get(name)
                    if word_gender is None:
                        word_gender = Genderize().get1(name).get('gender')
                        name_probability_list[name] = word_gender
                    if word_gender == 'male':
                        Male_count += 1
                    if word_gender== 'female':
                        Female_count += 1
                except Exception as exception:
                    APIcallfail +=1
            else: 
                sentence.append(name.strip()) #add a tokenize
    
#Lower
    sentence = [x.lower() for x in sentence]

#contractions
    new_text = []
    for word in sentence:
        contraction = contractions.get(word)
        if contraction is None:
            new_text.append(word)
        else:
            for word in contraction.split():
                new_text.append(word)

    sentence = new_text

#gendered_count
    for w in sentence:
        if w in male_list:
            Male_count += 1
        if w in female_list:
            Female_count += 1

#remove_stopwords
    stops = set(stopwords.words("english"))
    sentence = [x for x in sentence if not x in stops]

#remove_leakage
    new_sent = [x for x in sentence if x not in male_list]
    new_sent = [x for x in new_sent if x not in female_list]
    sentence = new_sent

    print(sentence)
    return sentence, Male_count, Female_count, APIcallfail

#if still having memory issues we needd to chunk the dataset 



In [ ]:
df_10['encoded_sentences'] = df_10['sentences'].apply(PreProcessing)

In [5]:
df_09_split = pd.DataFrame(df_09["encoded_sentences"].to_list(), columns=['pre_processed_sent','male_count','female_count','apicall_fail'])
result = pd.concat([df_09_split, (df_09.reset_index(drop=True))], axis=1)
result = result.drop("encoded_sentences", axis=1)


In [9]:
#save the api calls!!!
with open('name_probability_list.pickle', 'wb') as handle:
    pickle.dump(name_probability_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#save the df
with open('2009_preprocessed_date.pickle', 'wb') as handle:
    pickle.dump(df_09, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
#REFERENCE LISTS--TO BE CHANGED
male_list= {"abbot", "abbots", "actor", "actors", "author", "authors", "bachelor", "bachelors", "baron", "barons", "boy", "boys", "bridegroom", "bridegrooms", "brother", "brothers", "buck", 
            "bucks", "conductor", "conductors", "czar", "czars", "dad", "daddy", "daddys", "dads", "duke", "dukes", "emperor", "emperors", "enchanter", "enchanters", "father", "fathers", 
            "gentleman", "gentlemans", "granddad", "granddads", "grandfather", "grandfathers", "grandpa", "he", "heir", "heirs", "hero", "heros", "hes", "him", "his", "host", "hosts", "hunter", 
            "hunters", "husband", "husbands", "king", "kings", "landlord", "landlords", "lord", "lords", "man", "mans", "manservant", "manservants", "master", "masters", "men", "milkman", 
            "milkmans", "mister", "monk", "monks", "mr", "nephew", "nephews", "patron", "patrons", "peacock", "peacocks", "peer", "peers", "poet", "poets", "policeman", "policemans", "policemen", 
            "policemens", "priest", "priests", "prince", "princes", "prophet", "prophets", "shepherd", "shepherds", "signor", "signors", "sir", "sirs", "son", "songster", "songsters", "sons", 
            "stag", "stags", "stallion", "stallions", "stepbrother", "stepbrothers", "stepdad", "stepdads", "stepfather", "stepfathers", "steward", "stewards", "sultan", "sultans", "traitor", 
            "traitors", "uncle", "uncles", "viscount", "viscounts", "waiter", "waiters", "wizard", "wizards"}

female_list ={"abbess", "abbesses", "actress", "actresses", "aunt", "aunts", "authoress", "authoresses", "baroness", "baronesses", "benefactress", "benefactresses", "bride", "brides", 
              "conductress", "conductresses", "countess", "countesses", "czarina", "czarinas", "daughter", "daughters", "duchess", "duchesses", "empress", "empresses", "enchantress", 
              "enchantresses", "giantess", "giantesses", "girl", "girls", "goddess", "goddesses", "grandma", "grandmas", "grandmother", "grandmothers", "heiress", "heiresses", "hen", 
              "hens", "her", "heroine", "heroines", "hers", "hostess", "hostesses", "huntress", "huntresses", "ladies", "lady", "landladies", "landlady", "lioness", "lionesses", "madam", 
              "madams", "maidservant", "maidservants", "milkmaid", "milkmaids", "misses", "missus", "mistress", "mistresses", "mom", "mommy", "mommys", "moms", "mother", "mothers", "mrs", "ms",
                "mum", "mummy", "mummys", "mums", "murderess", "murderesses", "niece", "nieces", "nun", "nuns", "patroness", "patronesses", "poetess", "poetesses", "policewoman", "policewomen", 
                "priestess", "priestesses", "princess", "princesses", "prophetess", "prophetesses", "queen", "queens", "she", "shepherdess", "shepherdesses", "shes", "signora", "signoras", 
                "sister", "sisters", "songstress", "songstresses", "spinster", "spinsters", "stepdaughter", "stepdaughters", "stepmom", "stepmoms", "stepmother", "stepmothers", "stewardess", 
                "stewardesses", "sultana", "sultanas", "temptress", "temptresses", "tigress", "tigresses", "traitress", "traitresses", "viscountess", "viscountesses", "vixen", "vixens", 
                "waitress", "waitresses", "wife", "witch", "witches", "wive", "woman", "women"}

#should all gendered words be counted as a gendered thing???
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [10]:
result["year"].value_counts()

2009    9712
Name: year, dtype: int64

In [30]:
df_sample_1 = result.iloc[0:4875]
df_sample_2 = result.iloc[4876::]

In [34]:
df_sample_2 = df_sample_2.replace('2009','2000')

In [20]:
def compare_count(male_col, female_col): 
    """This function compares the count of female to male pronouns. It will output "1" if male count bigger
    than female count, "neutral" if the count is equal, and "female" if there is a higher female count. 
    The function returns strings because we need categorical variables for log reg to run"""
    if female_col > male_col:
        return "1"
    elif male_col > female_col:
        return "0"
    else: 
        return None

In [37]:
df_sample_1['col_type'] = df_sample_1.apply(lambda row: compare_count(row['male_count'], row['female_count']),axis=1)
df_sample_2['col_type'] = df_sample_2.apply(lambda row: compare_count(row['male_count'], row['female_count']),axis=1)



C:\Users\DanielleDuncan\AppData\Local\Temp\ipykernel_14360\2854444328.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample_1['col_type'] = df_sample_1.apply(lambda row: compare_count(row['male_count'], row['female_count']),axis=1)


In [ ]:
#remove columns with "None" in the col_type 
sentences_df = sentences_df[sentences_df["col_type"].notnull()]
#sentences_df

In [47]:
df_sample_1 = df_sample_1.dropna(subset = ["col_type"])
df_sample_2 = df_sample_2.dropna(subset = ["col_type"])
df_sample_2

,pre_processed_sent,male_count,female_count,apicall_fail,sentences,article_id,year,col_type
4878,"[start, new, weekly, slot, appear, watershed, ...",1,0,0,"The Bill is to start its new weekly slot, appe...",558151,2000,1
4885,"[play, say, new, post, watershed, slot, exciti...",1,0,1,"Sam Callis, who plays Sergeant Callum Stone, s...",558151,2000,1
4887,"[term, storytelling, opportunity, give, allow,...",1,0,0,"""In terms of the storytelling, and the opportu...",558151,2000,1
4888,"[still, get, flavour, tighter, sharper]",1,0,0,"""It's still The Bill, it's got that flavour to...",558151,2000,1
4891,"[play, say, drama, look, really, grown, quite,...",1,0,1,"""Alex Walkinshaw, who plays Smithy (Inspector ...",558151,2000,1
...,...,...,...,...,...,...,...,...
9706,"[organisers, say, extend, programme, live, tou...",2,0,0,Organisers of the X Factor have said they've e...,1043733,2000,1
9707,"[vote, judge, week, seven, competition]",1,0,0,Jedward were voted off by the X Factor judges ...,1043733,2000,1
9708,"[two, month, tour, begin, see, extra, date, add]",3,0,0,"The two month tour, which begins in Liverpool ...",1043733,2000,1
9709,"[artists, confirm, tour, contestant]",2,1,0,Artists confirmed for the tour are contestants...,1043733,2000,1


In [49]:
df_sample_1["col_type"].value_counts()

1    1177
0     630
Name: col_type, dtype: int64

In [ ]:
#class balancing and save as fully preprocessed. make it so the whole notebook runs per year.
#TODO seperate out the name function. save two copies, with names and ready for model. 

In [22]:
#YOLANDA SAMPLE DATA FOR LOG REG
result['col_type'] = result.apply(lambda row: compare_count(row['male_count'], row['female_count']),axis=1)
#results_df_09 = result[result["col_type"].notnull()]
sample_results_df_09 = result.dropna(subset = ["col_type"])
sample_results_df_09
sample_results_df_09.to_pickle('sample_results_df_09.pickle')